# Iterative fitting on Buckingham potential

In [2]:
import os
os.chdir("/home/thism292/Documents/repos/CCS_fit/examples/Advanced_Tutorials/Iterative_Fitting/Generate_Reference_Data/")

In [3]:
# Load in the relevant packages

from ase.io import read,write
from ase.build import bulk
import numpy as np
import ase.db as db
from ase.visualize import view
from ase.calculators.lj import LennardJones
import matplotlib.pyplot as plt
import json

from ccs_fit.ase_calculator.buck import Buck
with open("Buck_params.json", "r") as f: # From Calleja 2003 / https://iopscience.iop.org/article/10.1088/0953-8984/15/14/305/pdf
    Buck_params = json.load(f)

from ase.io import read

Fit_on_forces=False  #Enable/disable option for fitting CCS potential to atomic forces. 

### Generate training data
Curvature Constrained Splines can be fitted to a reference data-set with energies (and optionally forces) of pre-calculated structures. In this example we generate the reference data-set using a Lennard-Jones potential. We use a LJ bulk structure that we randomly rattle and scale. 


In [6]:
CaTiO3 = read('CaTiO3.poscar')
CaTiO3 = CaTiO3 * [4, 4, 3]

calc = Buck(Buck_params)
CaTiO3.calc = calc

orig_cell = CaTiO3.get_cell()
orig_struc = CaTiO3.copy()

displacement_magnitude=0.03
disp_steps=5
rounds=3

trainset_list="file_list" # List of strucutres to include in the database (see below)
f = open(trainset_list, "w")
counter=1
for round in range(rounds):
    for scale in np.linspace(0.95, 1.05, 6):
        new_cell = orig_cell*scale
        new_struc = orig_struc.copy()
        new_struc.set_cell(new_cell)
        new_struc.calc = calc
        nrg = new_struc.get_potential_energy()
        for i in range(disp_steps):
            rattle_struc = new_struc.copy()
            rattle_struc.rattle(displacement_magnitude*i, seed=counter)
            rattle_struc.calc = calc
            nrg = rattle_struc.get_potential_energy()
            print(nrg)
            xyz_file=f"CALCULATED_DATA/S{counter}.xyz"
            write(xyz_file,rattle_struc)
            print(xyz_file,file=f)
            counter += 1

f.close()
       

12546.360759021092
12808.910296457905
12802.789671708042
12881.920280195303
14161.763665346878
7296.989214311916
7373.403800753854
7581.666523449209
7894.171337854036
8187.723997356813
5395.985716527156
5436.513051162852
5556.075255551225
5812.027108912518
6115.882701756425
4636.1692147342865
4663.374071279288
4793.857097610353
4967.351421941416
5212.8779582066045
4289.755215688891
4334.646007988459
4431.468914756558
4591.52280023188
4864.215695341181
4126.122937479025
4164.797232817886
4289.275861004553
4431.272142977646
4613.170049895938
12546.360759021092
12405.542825437726
12596.61591090148
13932.804322469063
13080.888487890023
7296.989214311916
7331.563778411622
7454.333651559037
7842.886946277381
8230.55103256549
5395.985716527156
5434.679854391927
5559.514539956003
5792.066374971323
6087.621213817829
4636.1692147342865
4671.563849009286
4793.377661327943
4999.037186787964
5314.371818238511
4289.755215688891
4325.887447459358
4406.4739373893
4628.414749910102
4870.227445541399
41

In [7]:
from ccs_fit.scripts.ccs_build_db import ccs_build_db

ccs_build_db(mode="CCS",DFT_DB="CaTiO3.db",file_list="file_list",overwrite=True)

    Building data-bases: 100%|██████████████████████████████████| 90/90 [00:01<00:00, 86.90it/s]


### Generate the CCS_fit input file; structures.json
The next commands fetch the training set data from the ASE database containing the LJ structures and the corresponding energies. The reference data is called ``LJ.db`` since the reference energies and forces are obtained from a Lennard Jones potential.

In [8]:
from ccs_fit.scripts.ccs_fetch import ccs_fetch

ccs_fetch(mode="CCS",DFT_DB="CaTiO3.db",include_forces=Fit_on_forces)

    Fetching data: 100%|████████████████████████████████████████| 90/90 [00:51<00:00,  1.76it/s]


### Fit training data to Curvature Constrained Splines
Finally, the splines are fitted to the target defined in the structures.json input file. The splines can be restricted to be fully repulsive ("rep"), or have a turning point/switch ("sw"), which is defined by the "Swtype" key. A more comprehensive guide on the fitting options can be found in Advanced_Tutorials/Search_Mode.


In [9]:
### Generate input.json file
import json

input={
    "General": {
        "interface": "CCS",
        "merging"  : "True"
    },
    "Twobody": {
                "O-O": {
                        "Rcut": 6.0,
                        "Resolution": 0.01,
                        "Swtype": "sw",
                        "const_type" : "Mono"
                },
                "O-Ti": {
                        "Rcut": 6.0,
                        "Resolution": 0.01,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                },
                "Ti-Ti": {
                        "Rcut": 6.0,
                        "Resolution": 0.01,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                }
        }
}
#SAVE TO FILE
with open('CCS_input.json', 'w') as f:
    json.dump(input, f, indent=8)

In [10]:
### Generate input.json file
import json

input={
    "General": {
        "interface": "CCS",
        "merging"  : "True"
    },
    "Twobody": {
                "Ca-O": {
                        "Rcut": 6.0,
                        "Resolution": 0.05,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                },
                "O-O": {
                        "Rcut": 6.0,
                        "Resolution": 0.05,
                        "Swtype": "sw",
                        "const_type" : "Mono"
                },
                "Ti-O": {
                        "Rcut": 6.0,
                        "Resolution": 0.05,
                        "Swtype": "rep",
                        "const_type" : "Mono"
                }
        }
}

#SAVE TO FILE
with open('CCS_input.json', 'w') as f:
    json.dump(input, f, indent=8)

In [11]:
#RUN FIT
from ccs_fit import ccs_fit

ccs_fit("CCS_input.json")

    Generating one-body information from training-set.
        Added elements:  ['Ca', 'O', 'Ti']
    Applying monotonic constraints for pair:  Ca-O
    Applying monotonic constraints for pair:  O-O
    Applying monotonic constraints for pair:  Ti-O
    Merging intervals for pair Ca-O; number of intervals reduced from 106 to 104. 
    Merging intervals for pair Ti-O; number of intervals reduced from 114 to 113. 
    There is linear dependence in stochiometry matrix!
    Removing onebody term: O
    There is linear dependence in stochiometry matrix!
    Removing onebody term: Ti


    Finding optimum switch: 100%|███████████████████████████████| 97/97 [00:07<00:00, 12.33it/s]


    The best switch is (104, 16, 113) with rmse: 7.69499837556838e-08, corresponding to distances of [5.99 2.04 6.04] Å for element pairs ['Ca-O', 'O-O', 'Ti-O'].


### Validate your potential
Make sure your potential (at least) reproduce the data points in your training-set. Performing further tests on strucutres not included in the training set is recomended but not included in the tutorial.

In [ ]:
from ccs_fit.scripts.ccs_validate import ccs_validate
ccs_validate(mode="CCS",CCS_params="CCS_params.json",DFT_DB="CaTiO3.db")

In [ ]:
with open("CCS_params.json", "r") as f:
    CCS_params = json.load(f)

with open("structures.json", "r") as f:
    training_set = json.load(f)

r=np.array(CCS_params["Two_body"]["O-O"]["r"])
e=CCS_params["Two_body"]["O-O"]["spl_a"]
A = 22764 # 3242.124
B = 6.7114 # 3.4626
C = 27.88 # 0.
e_Buck = A*np.exp(-B*r) - C/r**6
plt.xlim(1.,3)
plt.ylim(-3,1)
plt.xlabel('Distance (Å)')
plt.ylabel('Potential (eV)')
plt.plot(r,e_Buck,color='black',label="Ref. Lennard-Jones potential")
plt.plot(r,e,'--',color='red',label="Fitted potential")
plt.legend()
plt.show()

err=np.loadtxt("CCS_validate.dat")
plt.xlabel('Reference energy (eV)')
plt.ylabel('Validation energy (eV)')
plt.plot( [min(err[:,0]),max(err[:,0])],[min(err[:,0]),max(err[:,0])],'--',color='black'  )
plt.scatter(err[:,0],err[:,1],facecolors='none', edgecolors='red')
plt.show()
plt.xlabel('Reference energy (eV)')
plt.ylabel('Error in fit (eV)')
plt.scatter(err[:,0],err[:,2],facecolors='none', edgecolors='red')
plt.show()

try:
    err_F=np.loadtxt("CCS_error_forces.out")
    plt.xlabel('Reference force (eV/Å)')
    plt.ylabel('Fitted force (eV/Å)')
    plt.plot( [min(err_F[:,0]),max(err_F[:,0])],[min(err_F[:,0]),max(err_F[:,0])],'--',color='black')
    plt.scatter(err_F[:,0],err_F[:,1],facecolors='none', edgecolors='red',alpha=0.1 )
    plt.show()
except:
    pass

d=[]
for t in training_set["energies"]:
    d.extend(training_set["energies"][t]["O-O"])
    
plt.hist(d)
plt.show()

In [ ]:
from ccs_fit.scripts.ccs_export_FF import write_FF

write_FF("CCS_params.json")

### Pruning

In [12]:
from shutil import copyfile
copyfile("CCS_params.json", "CCS_params_reference.json")

'CCS_params_reference.json'

In [15]:
!ccs_prune -d "CaTiO3.db"


--------------------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/thism292/anaconda3/envs/CCS_fiterative/bin/ccs_prune", line 6, in <module>
    sys.exit(main())
  File "/home/thism292/Documents/repos/CCS_fit/src/ccs_fit/scripts/ccs_prune.py", line 169, in main
    ccs_prune(**vars(args))
  File "/home/thism292/Documents/repos/CCS_fit/src/ccs_fit/scripts/ccs_prune.py", line 60, in ccs_prune
    with open(CCS_params, "r") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'CCS_params_pre_prune.json'


In [ ]:
from ccs_fit.scripts.ccs_build_db import ccs_build_db

ccs_build_db(mode="CCS",DFT_DB="TiO2.db",file_list="file_list",overwrite=True)

In [ ]:
TiO2 = read('TiO2.poscar')
TiO2 = TiO2 * [4,4,6]

calc = Buck(Buck_params)
TiO2.calc = calc

orig_cell = TiO2.get_cell()
orig_struc = TiO2.copy()

displacement_magnitude=0.03
disp_steps=5
rounds=5

trainset_list="file_list" # List of strucutres to include in the database (see below)
f = open(trainset_list, "w")
counter=1
for round in range(rounds):
    for scale in np.linspace(0.95, 1.05, 6):
        new_cell = orig_cell*scale
        new_struc = orig_struc.copy()
        new_struc.set_cell(new_cell)
        new_struc.calc = calc
        nrg = new_struc.get_potential_energy()
        for i in range(disp_steps):
            rattle_struc = new_struc.copy()
            rattle_struc.rattle(displacement_magnitude*i, seed=counter)
            rattle_struc.calc = calc
            nrg = rattle_struc.get_potential_energy()
            print(nrg)
            xyz_file=f"CALCULATED_DATA/S{counter}.xyz"
            write(xyz_file,rattle_struc)
            print(xyz_file,file=f)
            counter += 1

f.close()

### Cleaning up

In [ ]:
import glob
import os

def rm(file):
    try:
        os.remove(file)
    except OSError:
        pass


list=glob.glob("CALCULATED_DATA/*")
for file in list:
    rm(file)
list=glob.glob("CCS_*")
for file in list:
    rm(file)
rm("structures.json")
rm("file_list")
rm("LJ.db")
rm("ccs.spl")
rm("Buckingham.dat")
rm("Lennard_Jones.dat")
rm("Morse.dat")
rm("Pedone.dat")
rm("CCS.lammps")